In [ ]:
import sys
from IPython.display import Markdown
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["DEEPSEEK_API_KEY", "LANGSMITH_ENDPOINT", "LANGSMITH_API_KEY", "LANGSMITH_PROJECT"])

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E 'lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

LANGSMITH_ENDPOINT密钥已加密保存
LANGSMITH_API_KEY密钥已加密保存
LANGSMITH_PROJECT密钥已加密保存
Python 3.10.16
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        6.3.7
langchain                                0.3.19
langchain-chroma                         0.2.2
langchain-community                      0.3.18
langchain-core                           0.3.49
langchain-deepseek                       0.1.3
langchain-openai                         0.3.11
langchain-text-splitters                 0.3.6
langgraph                                0.3.21
langgraph-checkpoint                     2.0.23
langgraph-prebuilt                       0.1.7
langgraph-sdk                            0.1.60
langserve                                0.3.1
langsmith                                0.3.8
numpy                                    1.26.4
openai                                   1.69.0
scipy                                    1.15.2
tikto

# 构建一个提取链

在本教程中，我们将利用[聊天模型](/docs/concepts/chat_models)的[工具调用功能](/docs/concepts/tool_calling)，从非结构化文本中提取结构化信息。同时，我们还将展示如何在此场景下运用[少量样本提示技术](/docs/concepts/few_shot_prompting/)来提升性能。

:::重要
本教程需要 `langchain-core>=0.3.20` 版本，且仅支持具备**工具调用**功能的模型。
:::

## 安装设置

### Jupyter Notebook

本教程及其他教程或许在 [Jupyter notebooks](https://jupyter.org/) 中运行最为便捷。在交互式环境中学习指南是深入理解它们的绝佳方式。安装方法请参阅 [此处](https://jupyter.org/install)。

### 安装

要安装LangChain，请运行：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
<CodeBlock language="bash">pip install --upgrade langchain-core</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
<CodeBlock language="bash">conda install langchain-core -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。
随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。
最佳实践是使用 [LangSmith](https://smith.langchain.com)。

在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```
```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

In [2]:
import os

os.environ['LANGSMITH_TRACING'] = "true"

## 架构

首先，我们需要描述要从文本中提取哪些信息。

我们将使用 Pydantic 来定义一个示例模式以提取个人信息。

In [3]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """关于一个人的信息。"""

    # ^ 人物Person的实体的文档字符串。
    # 此文档字符串将作为模式Person的描述发送到LLM，
    # 并且可以帮助改善提取结果。

    # 请注意：
    # 1. 每个字段都是`optional` -- 这允许模型拒绝提取它！
    # 2. 每个字段都有一个`description` -- 此描述由LLM使用。
    # 有一个好的描述可以帮助改善提取结果。
    name: Optional[str] = Field(default=None, description="人的名字")
    hair_color: Optional[str] = Field(
        default=None, description="如果已知，人的头发颜色"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="以米为单位的身高"
    )

定义模式时的两个最佳实践：

1. 记录**属性**及**模式**本身：这些信息会被发送至大语言模型（LLM），用于提升信息提取的质量。
2. 不要强迫大语言模型编造信息！如上所述，我们对属性使用了`Optional`，允许大语言模型在不知道答案时输出`None`。

:::重要
为了获得最佳性能，请详细记录模式，并确保在没有需要提取的信息时，模型不会被强制返回结果。
:::

## 提取器

让我们使用上面定义的架构来创建一个信息提取器。

In [4]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# 定义一个自定义的提示，以提供指令和任何额外的上下文。
# 1) 您可以将示例添加到提示模板中，以提高提取质量
# 2) 引入额外的参数，以考虑上下文（例如，包括文本被提取的文档的元数据）。
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "您是一个专门的提取算法。 "
            "只从文本中提取相关信息。 "
            "如果您不知道要提取的属性的值， "
            "则返回该属性值为null。",
        ),
        # 请参阅关于使用参考示例提高性能的方法。
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

我们需要使用一个支持函数/工具调用的模型。

请查阅[相关文档](/docs/concepts/tool_calling)了解所有可与此API搭配使用的模型。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [5]:
# | output: false
# | echo: false

from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(model='deepseek-chat')

In [6]:
structured_llm = llm.with_structured_output(schema=Person)

让我们来测试一下：

In [7]:
text = "Alan Smith 身高6英尺，头发是金发的。"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='金发', height_in_meters=None)

:::重要

提取即生成 🤯

大型语言模型（LLMs）是生成式模型，因此它们能完成一些相当酷的任务，例如准确提取以米为单位的人体身高数据。
尽管它是以英尺为单位提供的！
:::

我们可以在此处查看LangSmith追踪记录[链接](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r)。需要注意的是，[追踪记录中的聊天模型部分](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r/dd1f6305-f1e9-4919-bd8f-339d03a12d01)显示了发送给模型的消息确切序列、调用的工具以及其他元数据。

## 多重实体

在**大多数情况下**，你应该提取一个实体列表而非单个实体。

通过使用pydantic将模型相互嵌套，可以轻松实现这一点。

In [8]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """关于一个人的信息。"""

    # ^ 对实体Person的文档字符串。
    # 这个文档字符串作为Person模式的描述被发送给LLM，
    # 它可以帮助提高提取结果。

    # 注意：
    # 1. 每个字段都是`可选的` -- 这允许模型拒绝提取它！
    # 2. 每个字段都有一个`描述` -- 这个描述被LLM使用。
    # 拥有一个好的描述可以帮助提高提取结果。
    name: Optional[str] = Field(default=None, description="人的姓名")
    hair_color: Optional[str] = Field(
        default=None, description="如果已知，人的头发颜色"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="以米为单位测量的身高"
    )


class Data(BaseModel):
    """关于人们的提取数据。"""

    # 创建一个模型，以便我们可以提取多个实体。
    people: List[Person]

:::重要
提取结果在此可能并不完美。继续阅读以了解如何使用**参考示例**来提高提取质量，并查看我们的提取[操作指南](/docs/how_to/#extraction)以获取更多详细信息。

In [9]:
structured_llm = llm.with_structured_output(schema=Data)
text = "我的名字是杰夫，我的头发是黑色的，我6英尺高。安娜的头发颜色和我一样。"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='杰夫', hair_color='黑色', height_in_meters='1.8288'), Person(name='安娜', hair_color='黑色', height_in_meters=None)])

In [ ]:
text = "我是Rookie, 我的头发是黑色的。我正在和250cm高的Jack学习LangChain"
chain =  prompt_template | structured_llm
chain.invoke({"text": text})

Data(people=[Person(name='Rookie', hair_color='黑色', height_in_meters=None), Person(name='Jack', hair_color=None, height_in_meters='250')])

:::提示
当模式支持提取**多个实体**时，若不存在相关信息，该模式也允许模型**不提取任何实体**。
在文本中通过提供一个空列表来表示。

这通常是一件**好**事！它允许在实体上指定**必需**的属性，而不必强制模型检测该实体。
:::

我们可以在[这里](https://smith.langchain.com/public/7173764d-5e76-45fe-8496-84460bd9cdef/r)查看LangSmith的追踪记录。

## 参考示例

可以通过[少量样本提示](/docs/concepts/few_shot_prompting/)来引导LLM应用的行为。对于[聊天模型](/docs/concepts/chat_models/)而言，这可以表现为一系列输入与响应消息的配对组合，用以展示期望的行为模式。

例如，我们可以通过交替使用 `user` 和 `assistant` 的[消息](/docs/concepts/messages/#role)来传达符号的含义：

In [14]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},  
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7  

The parrot (🦜) seems to be acting as a plus sign (+) here!  

So:  
- 2 🦜 2 = 2 + 2 = **4**  
- 2 🦜 3 = 2 + 3 = **5**  
- 3 🦜 4 = 3 + 4 = **7**  

Fun pattern! 😊


[结构化输出](/docs/concepts/structured_outputs/)通常底层使用了[工具调用](/docs/concepts/tool_calling/)功能。这一过程通常涉及生成包含工具调用的[AI消息](/docs/concepts/messages/#aimessage)，以及包含工具调用结果的[工具消息](/docs/concepts/messages/#toolmessage)。在这种情况下，消息序列应该是怎样的？

不同的[聊天模型提供商](/docs/integrations/chat/)对有效消息序列有着不同的要求。有些会接受如下形式的（可重复）消息序列：

- 用户消息
- 带工具调用的AI消息
- 附带结果的工具消息

其他情况下则需要一条包含某种回应的最终AI消息。

LangChain 包含一个实用函数 [tool_example_to_messages](https://python.langchain.com/api_reference/core/utils/langchain_core.utils.function_calling.tool_example_to_messages.html)，该函数可为大多数模型提供商生成有效的消息序列。它通过仅需对应工具调用的 Pydantic 表示形式，简化了结构化小样本示例的生成过程。

让我们来试试这个方法。我们可以将输入字符串与期望的Pydantic对象配对，转换成可提供给聊天模型的消息序列。在底层实现中，LangChain会将工具调用格式化为每个供应商所需的格式。

注意：此版本的 `tool_example_to_messages` 需要 `langchain-core>=0.3.20`。

In [15]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "海洋是广阔而蓝的。它超过20,000英尺深。",
        Data(people=[]),
    ),
    (
        "菲奥娜从法国远行到西班牙。",
        Data(people=[Person(name="菲奥娜", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # 这条最终消息对于一些提供商来说是可选的
        ai_response = "检测到人。"
    else:
        ai_response = "未检测到人。"
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

C:\Users\19145\AppData\Local\Temp\ipykernel_6424\2093107491.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


检查结果时，我们发现这两个示例对生成了八条消息：

In [16]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

海洋是广阔而蓝的。它超过20,000英尺深。
================================== Ai Message ==================================
Tool Calls:
  Data (a32db15f-180b-4bf9-a519-3b75d5a521b0)
 Call ID: a32db15f-180b-4bf9-a519-3b75d5a521b0
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

未检测到人。
================================ Human Message =================================

菲奥娜从法国远行到西班牙。
================================== Ai Message ==================================
Tool Calls:
  Data (7d78a15f-45d2-4289-83a1-9775ba3d1e15)
 Call ID: 7d78a15f-45d2-4289-83a1-9775ba3d1e15
  Args:
    people: [{'name': '菲奥娜', 'hair_color': None, 'height_in_meters': None}]
================================= Tool Message =================================

You have correctly called this

让我们对比一下有这些消息和没有这些消息时的性能表现。例如，我们传递一条本意不提取任何人名的消息：

In [17]:
message_no_extraction = {
    "role": "user",
    "content": "太阳系很大，但地球只有1个月亮。",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color=None, height_in_meters=None)])

在此示例中，模型容易错误地生成人物记录。

由于我们的少量示例中包含“负面”案例，我们鼓励模型在这种情况下做出正确反应。

In [18]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

:::提示

[LangSmith](https://smith.langchain.com/public/b3433f57-7905-4430-923c-fed214525bf1/r) 运行轨迹揭示了发送至聊天模型的具体消息序列、生成的工具调用、延迟时间、令牌计数以及其他元数据。

:::

请参阅[本指南](/docs/how_to/extraction_examples/)了解包含参考示例的提取工作流程详情，其中涵盖如何整合提示模板及自定义示例消息的生成方法。

## 后续步骤

既然你已经掌握了使用LangChain进行信息提取的基础知识，接下来可以继续学习其余的操作指南：

- [添加示例](/docs/how_to/extraction_examples): 关于使用**参考示例**提升性能的更多细节。
- [处理长文本](/docs/how_to/extraction_long_text)：如果文本超出大语言模型的上下文窗口限制，该如何应对？
- [采用解析方法](/docs/how_to/extraction_parse)：对于不支持**工具/函数调用**的模型，使用基于提示词的方法进行信息提取。